# Cost of Living Datenanalyse - Jupyter Notebook mit API-Abfrage

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import re

In [42]:
API_KEY = "8773|XHzf41HtmUEBgTLSbetLG8rMDkKznRmYk5JGviM9"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

cities = [
    {"city": "vienna", "country": "austria"},
    {"city": "graz", "country": "austria"},
    {"city": "linz", "country": "austria"},
    {"city": "salzburg", "country": "austria"},

    {"city": "berlin", "country": "germany"},
    {"city": "munich", "country": "germany"},
    {"city": "hamburg", "country": "germany"},
    {"city": "dusseldorf", "country": "germany"},
    {"city": "frankfurt", "country": "germany"},
    {"city": "hannover", "country": "germany"},
    {"city": "nuremberg", "country": "germany"},
    {"city": "essen", "country": "germany"},
    {"city": "cologne", "country": "germany"},

    {"city": "bern", "country": "switzerland"},
    {"city": "zurich", "country": "switzerland"},
    {"city": "geneva", "country": "switzerland"},
    {"city": "basel", "country": "switzerland"}
]

base_url = "https://zylalabs.com/api/3440/world+cost+of+living+api/3745/get+prices"

results = []

def normalize_keys(data):
    normalized = {}
    for k, v in data.items():
        key = k.lower()
        key = re.sub(r"[^a-z0-9]+", "_", key)
        key = key.strip("_")
        normalized[key] = v
    return normalized

for loc in cities:
    params = {"city": loc["city"], "country": loc["country"]}
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        raw = response.json()
        norm = normalize_keys(raw)
        norm["city_name"] = loc["city"]
        norm["timestamp"] = time.time()
        results.append(norm)
    else:
        print(f"Fehler bei {loc['city']}: {response.status_code}")

### 2. Umwandlung in Dataframe

In [46]:
df = pd.DataFrame(results)
print(df.columns.tolist())

### 3. Daten vorbereiten

In [47]:
for col in [
    "average_monthly_net_salary_after_tax",
    "apartment_1_bedroom_in_city_centre",
    "meal_inexpensive_restaurant",
    "internet_60_mbps_or_more_unlimited_data_cable_adsl",
    "basic_electricity_heating_cooling_water_garbage_for_915_sq_ft_apartment"
]:
    if col in df.columns:
        df[col] = df[col].replace({"€": "", ",": ""}, regex=True)
        df[col] = pd.to_numeric(df[col], errors="coerce")

### 4. Übersicht anzeigen

In [48]:
df_overview = df[[
    "city_name",
    "average_monthly_net_salary_after_tax",
    "apartment_1_bedroom_in_city_centre",
    "meal_inexpensive_restaurant",
    "internet_60_mbps_or_more_unlimited_data_cable_adsl"
]]
df_overview = df_overview.dropna()

### 5. Visualisierung

In [39]:
df.head()

,version,status,success,city_name,meal_inexpensive_restaurant,meal_for_2_people_mid_range_restaurant_three_course,mcmeal_at_mcdonalds_or_equivalent_combo_meal,domestic_beer_1_pint_draught,imported_beer_12_oz_small_bottle,cappuccino_regular,...,1_pair_of_men_leather_business_shoes,apartment_1_bedroom_in_city_centre,apartment_1_bedroom_outside_of_centre,apartment_3_bedrooms_in_city_centre,apartment_3_bedrooms_outside_of_centre,price_per_square_feet_to_buy_apartment_in_city_centre,price_per_square_feet_to_buy_apartment_outside_of_centre,average_monthly_net_salary_after_tax,mortgage_interest_rate_in_percentages_yearly_for_20_years_fixed_rate,timestamp
0,1,200,True,vienna,NaN,70.00 €,10.00 €,5.00 €,1.76 €,4.22 €,...,136.00 €,NaN,749.05 €,"1,982.12 €","1,420.31 €",884.51 €,553.32 €,NaN,3.63,1.750505e+09
1,1,200,True,berlin,NaN,70.00 €,12.00 €,4.50 €,1.65 €,3.77 €,...,121.59 €,NaN,900.04 €,"2,228.95 €","1,590.86 €",714.41 €,525.05 €,NaN,3.84,1.750505e+09
2,1,200,True,bern,NaN,87.50 Fr.,14.00 Fr.,6.75 Fr.,1.87 Fr.,4.69 Fr.,...,187.00 Fr.,NaN,"1,033.33 Fr.","2,637.71 Fr.","1,942.86 Fr.","1,111.11 Fr.",805.93 Fr.,NaN,2.52,1.750505e+09
